# Livneh Wrfcf Bias Tmin 2017-02-05 to 2017-02-17

In [1]:
# imports
import numpy
import Nio
import Ngl
from IPython import display
import netCDF4 as nc
from datetime import date
from datetime import timedelta
import xarray as xr
import xesmf as xe
import pandas as pd

In [2]:
# read in daily difference datasets
dif_05 = xr.open_dataarray('./dif_05_tmin-Copy1.nc')
dif_06 = xr.open_dataarray('./dif_06_tmin-Copy1.nc')
dif_07 = xr.open_dataarray('./dif_07_tmin-Copy1.nc')
dif_08 = xr.open_dataarray('./dif_08_tmin-Copy1.nc')
dif_09 = xr.open_dataarray('./dif_09_tmin-Copy1.nc')
dif_10 = xr.open_dataarray('./dif_10_tmin-Copy1.nc')
dif_11 = xr.open_dataarray('./dif_11_tmin-Copy1.nc')
dif_12 = xr.open_dataarray('./dif_12_tmin-Copy1.nc')
dif_13 = xr.open_dataarray('./dif_13_tmin-Copy1.nc')
dif_14 = xr.open_dataarray('./dif_14_tmin-Copy1.nc')
dif_15 = xr.open_dataarray('./dif_15_tmin-Copy1.nc')
#dif_16 = xr.open_dataarray('./dif_16_prev-Copy1.nc')
#dif_17 = xr.open_dataarray('./dif_17-Copy1.nc')

# define dictionary
#dif_dict = {5:dif_05, 6:dif_06, 7:dif_07, 8:dif_08, 9:dif_09, 
            #10:dif_10, 11:dif_11, 12:dif_12, 13:dif_13, 14:dif_14, 
            #15:dif_15, 16:dif_16, 17:dif_17}
dif_dict = {5:dif_05, 6:dif_06, 7:dif_07, 8:dif_08, 9:dif_09, 
            10:dif_10, 11:dif_11, 12:dif_12, 13:dif_13, 14:dif_14, 
            15:dif_15}

FileNotFoundError: [Errno 2] No such file or directory: b'/home/eudoraf12/Livneh Wrfcf NG-GR-NQ-BD/Livneh Wrfcf Tmax, Tmin 2017-02 Bias and RMSE/dif_05_tmin_prev-Copy1.nc'

In [ ]:
dif_dict[6].data.flatten()[155925]

In [ ]:
# average the difference over the 13 days, returns DataArray object
def bias_arr():
    next_key = 6
    temp = dif_dict[5].data.flatten()
    while next_key <= 15: #17
        arr1 = temp
        arr2 = dif_dict[next_key].data.flatten()
        next_key += 1
        temp = numpy.add(arr1, arr2)
    bias = temp / 11 #13
    return bias

bias_out = bias_arr()
bias_out = numpy.array_split(bias_out, 444)
lat_coord = dif_dict[6]['lat'].data.flatten()
lon_coord = dif_dict[6]['lon'].data.flatten()
bias_out = xr.DataArray(bias_out, coords = [lat_coord, lon_coord], dims = ['lat', 'lon'])
bias_out

In [ ]:
# some values for reference
numpy.unique(bias_out), max(numpy.unique(bias_out)), min(numpy.unique(bias_out))

In [ ]:
# save as a nc file for later
bias_out.to_netcdf('./bias_17_tmin.nc')

In [ ]:
# test plot
bias_out.plot()

In [ ]:
# convert dif_arr to masked array to overlook NaNs
import numpy.ma as ma
m_out = ma.masked_invalid(bias_out.data)
m_out

In [ ]:
# numerical data, mean bias over entire region
numpy.mean(m_out)

In [ ]:
# workstation
wks_type2 = "png"
wks2 = Ngl.open_wks(wks_type2,"17_bias_tmin_NG-GR-NQ-BD")

In [ ]:
# borrow RHS plot end coords
fn = '/cw3e/mead/projects/cwp103/production/output/2017020500/cf/NG-GR-NQ-BD/wrfcf_d02_2017-02-05_22_00_00.nc'
da_input = xr.open_dataset(fn)

In [ ]:
# defining variables
temp = m_out
lat = bias_out['lat'].data.flatten()
lon = bias_out['lon'].data.flatten()
lat2 = da_input['lat'].data.flatten()
lon2 = da_input['lon'].data.flatten()
tmax_nlat = len(lat)
tmax_nlon = len(lon)

In [ ]:
# XY of overlay plot
resources = Ngl.Resources()
resources.sfXCStartV = min(lon)
resources.sfXCEndV   = max(lon)
resources.sfYCStartV = min(lat)
resources.sfYCEndV   = max(lat)

# fill missing values as needed
if hasattr(temp,"_FillValue"):
    resources.sfMissingValueV = temp[0]._FillValue

# color map (sensitive to smaller values)
cmap = ['Ghostwhite', 'Grey0', 
        'Purple4', 'RoyalBlue3', 'RoyalBlue1', 'DeepSkyBlue', 'Cyan', 'DarkTurquoise', 
        'SeaGreen2', 'SpringGreen', 'Chartreuse', 'GreenYellow', 'OliveDrab2', 'Yellow', 
        'Gold', 'Orange', 'DarkOrange', 'Red', 'Red']

# plot styling
resources.wkColorMap = cmap
Ngl.set_values(wks2,resources)
resources.cnLevelSelectionMode = "ManualLevels"
resources.cnMinLevelValF       = -7
resources.cnMaxLevelValF       = 7
resources.cnLevelSpacingF      = 1
resources.tiXAxisString = "~F25~Longitude"
resources.tiYAxisString = "~F25~Latitude"
resources.cnFillOn              = True    
resources.cnLineLabelsOn        = False  
resources.cnInfoLabelOn         = False  
resources.nglSpreadColorEnd     = -2
resources.mpProjection = "LambertEqualArea" 
resources.mpCenterLonF = (lon[tmax_nlon-1] + lon[0])/2
resources.mpCenterLatF = (lat[tmax_nlat-1] + lat[0])/2
resources.mpLimitMode = "LatLon"
resources.mpMinLonF   = min(lon2)
resources.mpMaxLonF   = max(lon2)
resources.mpMinLatF   = min(lat2)
resources.mpMaxLatF   = max(lat2)
resources.mpOutlineBoundarySets = "GeophysicalAndUSStates"
resources.mpPerimOn   = True
resources.cnLinesOn   = False
resources.pmTickMarkDisplayMode = "Never"
resources.tiMainString = "~F26~Wrfcf, Livneh Bias Tmin, 2017-02-05 to 2017-02-17, NG-GR-NQ-BD"
resources.lbOrientation  = "Horizontal"
resources.lbTitleString  = "~F25~Temp (C)" 

In [ ]:
# plot
map = Ngl.contour_map(wks2,temp,resources)

In [ ]:
# clean up
del map
del resources
Ngl.end()

In [ ]:
# display plot
display.Image("./17_bias_tmin_NG-GR-NQ-BD.png")